# Introdução ao BERT (incompleto)

Neste notebook você verá um exemplo de como usar o [BERT](https://arxiv.org/abs/1810.04805) para extrair os embeddings de sentenças, além de conhecer mais sobre este modelo. 

Este tutorial foi adapato de:  BramVanroy / bert-for-inference (https://github.com/BramVanroy/bert-for-inference).

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

## O tokenizador (tokenizer)

Os modelos deep learning trabalham com tensors, que são basicamente vetores, que por sua vez são um grupo de números. Para começar, o texto de entrada (string) precisa ser convertido em um tipo de data que os modelos possam usar (números). Essa é a tarefa do tokenizador. 

In [ ]:
# Inicializando o tokenizador com um modelo pré-treinado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Durante o pré-treinamento, o tokenizador também é treinado, gerando um vocabulário conhecido. Cada palavra do vocabulário é atribuída a um índice (número), que pode ser usado pelo modelo. 

Para lidar com problemas das palavras que o tokenizador não conhece (out-of-vocabulary ou OOV), uma técnica é utilizada para garantir que o tokenizador aprenda "subpalavras". Desta forma, quando usamos os modelos pré-treinados, não teremos problemas de OOV. Quando o tokenizador não reconhece uma palavra, que não está no vocabulário, ele divide a palavra em pequenas unidades que são conhecidas. O tokenizador do BERT usa [WordPiece](https://arxiv.org/pdf/1609.08144.pdf) para dividir os tokens.

In [ ]:
# Converte a string "granola bars" para um vocabulário tokenizado 
granola_ids = tokenizer.encode('granola bars')
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))
# Converte os IDs to the actual vocabulary item
# Notice how the subword unit (suffix) starts with "##" to indicate 
# that it is part of the previous string
print('granola_tokens', tokenizer.convert_ids_to_tokens(granola_ids))

Você deve ter notado os tokens especiais [CLS] e [SEP]. Esses tokens são adicionados automaticamente pelo método `.encode()`, então não precisamos nos preocupar com eles. O primeiro é um token de classificação que foi pré-treinado, utilizado nas tarefas de classificação. Desta forma, ao invés de fazer a média de todos os tokens e usá-los como uma representação de frase, é recomendado apenas pegar a saída do [CLS] que representa a frase inteira. [SEP], por sua vez, é inserido como um separador entre várias instâncias, usado por exemplo na predição da próxima sentença, separando uma frase da outra.

Como vimos acima, o tipo de dados dos IDs de cada token é uma lista de inteiros. Neste notebook vamos usar a biblioteca `transformers` em combinação com PyTorch, que trabalha com tensores. Um tensor é um tipo especial de lista otimizada, normalmente usado em deep learning. Para converter nossos IDs dos tokens em um tensor, podemos simplesmente chamar um construtor de tensor passando a lista. Aqui, vamos usar um `LongTensor` que é usado para inteiros (para números de ponto flutuante,  usaríamos um `FloatTensor` ou apenas` Tensor`). 

O método `.encode ()` do tokenizer pode retornar um tensor em vez de uma lista, passando o parâmetro `return_tensors = 'pt'`, mas para fins de ilustração, faremos a conversão de uma lista para um tensor manualmente.

In [ ]:
# Convert the list of IDs to a tensor of IDs 
granola_ids = torch.LongTensor(granola_ids)
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))

## O modelo
Agora que pré-processamos nosso texto de entrada em um tensor de IDs (lembrando que cada valor de ID corresponde ao ID do token no vocabulário criado pelo tokenizador ), podemos alimentar o modelo. O modelo sabe qual palavra está sendo processada porque ele sabe qual token pertence a determinado ID. 

No BERT, assim como na maioria dos modelos de linguagem baseados em Transformers, a primeira camada é uma camada de embedding, cada token possui um embedding relacionado. No BERT, o embedding de um token é a soma de três tipos de embeddings: o embedding do token (gerado para o próprio token), o embedding do segmento (indica se o segmento faz parte da primeira ou o segunda sentença, não usado na inferência de uma única sentença) e o embedding de posição (distingue a posição do token na sentença). 

Para mais detalhes, veja [esse artigo](https://medium.com/@_init_/why-bert-has-3-embedding-layers-and-their-implementation-details-9c261108e28a). 

Abaixo, uma imagem do BERT retirada do artigo publicado.

![BERT embeddings visualization](img/bert-embeddings.png)

Para mais explicações sobre o modelo BERT, acesse [Jay Alammar's homepage](http://jalammar.github.io/).

Para começar, primeiro precisamos inicializar o modelo. Assim como o tokenizer, o modelo é pré-treinado, o que nos permite usar um modelo de linguagem já pré-treinado para obter representações de token ou de senteças.

Devemos usar o mesmo modelo pré-treinado que o tokenizer usa (`bert-base-uncased`). Este é o menor modelo BERT, treinado em texto em letras minúsculas. Desta forma, o tokenizer coloca o texto automaticamente em minúsculas para nós. A escolha do modelo, se deve ser caseado ou não, depende da tarefa. Tarefa como NER, por exemplo, podem requerer modelos treinados com maiúsculas e minúsculas.

No exemplo abaixo, um argumento adicional foi fornecido para a inicialização do modelo. `output_hidden_states` fornece mais informações de saída. Por padrão, um `BertModel` irá retornar uma tupla, mas o conteúdo dessa tupla é diferente dependendo da configuração do modelo. Ao passar `output_hidden_states = True`, a tupla irá conter:

1. O último estado oculto `(batch_size, sequence_length, hidden_size)`
2. pooler_output do token de classificação `(batch_size, hidden_size)`
3. os estados_ocultos das saídas do modelo em cada camada e as saídas dos embeddings iniciais
   `(batch_size, sequence_length, hidden_size)`

As placas gráficas (GPUs) são muito melhores em fazer operações em tensores do que uma CPU, portanto, sempre que disponível, executaremos os cálculos em GPU, como a CUDA (para isso, precisaremos de uma versão torch compatível com GPU.) 

Assim, movemos nosso modelo para o dispositivo correto: se estiver disponível, moveremos o modelo `.to ()` à GPU, caso contrário, permanecerá na CPU. É importante lembrar que o modelo e os dados a serem processados precisam estar no mesmo dispositivo. 

Finalmente, definimos o modelo para o modo de avaliação (`.eval`), em contraste com o modo de treinamento (` .train () `). Na avaliação, não temos por exemplo o dropout. 

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
# Set the device to GPU (cuda) if available, otherwise stick with CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()